In [3]:
%load_ext autoreload
%autoreload 2

from enumerate_datasets import *
from import_csv import *

In [5]:
dataset_paths = enumerate_datasets('.')
print('%d datasets found' %len(dataset_paths))

2 datasets found


In [6]:
for path in dataset_paths:
    import_csv(path)

In [12]:
#calculate grand mean
accumulator = 0;
count = 0;

for path in dataset_paths:
    print path
    con = sqlite3.connect(path.replace('.csv', '.db'))
    c = con.cursor()
    c.execute("""SELECT result FROM data""")
    for value in c.fetchall():
        print value[0]
        accumulator += value[0]
        count += 1

gm = accumulator / count
print ('grand mean is %f' %gm)

./op1.csv
-5.528
1.566
-3.805
5.464
6.756
-9.071
11.635
-11.929
8.022
-7.669
./op2.csv
-5.528
1.566
-3.805
5.464
6.756
-9.071
11.635
-11.929
8.022
-7.669
grand mean is -0.455900


In [ ]:
for value in c.fetchall():
    print value

In [ ]:
#define interface at this step